### Import library

In [1]:
import ace_lib as ace
import helpful_functions as hf
import pandas as pd
import requests
import plotly.express as px
import pygwalker as pyg
import glob
import re

### Start session
Enter credentials once - they will be saved to local folder and loaded each time

In [2]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_pGgeKKQm3HUomWv1dLUCb2uhTPef

 


## Scoring

In [104]:
def score(raw_data):
    score_data = raw_data[['test.turnover', 'test.returns', 'test.drawdown']]
    score = pd.DataFrame()
    score = 1.3673 + 0.1757*score_data['test.turnover'] + 0.6150*score_data['test.returns'] - 1.0816*score_data['test.drawdown']
    return score

## Alpha Mixing Templates

In [18]:
'''
signal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));

signal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));

signal1 * (1 + signal2)
'''

'\nsignal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));\n\nsignal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));\n\nsignal1 * (1 + signal2)\n'

In [19]:
'''
signal1 = oth176_lmt_close_eq_score ;
signal2 = oth176_malta_eq_score ;
k = ts_corr(signal1, signal2, 30) ;
k*regression_neut(signal1, signal2)

[아이디어]
경향성이 비슷한 알파 활용하여 corr로 weight를 부여
'''

'\nsignal1 = oth176_lmt_close_eq_score ;\nsignal2 = oth176_malta_eq_score ;\nk = ts_corr(signal1, signal2, 30) ;\nk*regression_neut(signal1, signal2)\n\n[아이디어]\n경향성이 비슷한 알파 활용하여 corr로 weight를 부여\n'

## Read all passed alphas

In [2]:
file_direc = glob.glob('Files/signal/signal_form/*')
file_direc

['Files/signal/signal_form\\A_signalform.csv',
 'Files/signal/signal_form\\B_signalform.csv',
 'Files/signal/signal_form\\C_signalform.csv',
 'Files/signal/signal_form\\D_signalform.csv']

In [3]:
expressions = {}
expressions['A'] = pd.read_csv(file_direc[0])['expression']
expressions['B'] = pd.read_csv(file_direc[1])['expression']
expressions['C'] = pd.read_csv(file_direc[2])['expression']
expressions['D'] = pd.read_csv(file_direc[3])['expression']

## Template C mixing
use top 3 result

In [6]:
#region= "USA", universe = "TOP3000", neutralization = 'FAST')

In [5]:
c = expressions['C'][:3]
len(c)

3

In [ ]:
'''
signal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));

signal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));

signal1 * (1 + signal2)
'''

### add

In [14]:
alpha_list = []

for i in range(2):
    alpha_list.append(ace.generate_alpha(f'{c[2*i]};{c[2*i+1]};zscore(add(signal_C_{2*i}, signal_C_{2*i+1}, filter = false))', region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.01))

alpha_list

[{'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 0,
   'neutralization': 'MARKET',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': 'pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21);factor_A_0 = group_zscore(ts_ir(returns, 252), industry);alpha_A_0 = regression_neut(stock_A_0, factor_A_0);signal_A_0 = trade_when(pro_A_0<=0.5, alpha_A_0, pro_A_0>0.5);pro_A_1 = oth460_1l_ccns;stock_A_1 = ts_zscore(ts_backfill(anl69_cpss_best_eeps_nxt_yr, 21),21);factor_A_1 = group_zscore(ts_ir(returns, 252), industry);alpha_A_1 = regression_neut(stock_A_1, factor_A_1);signal_A_1 = trade_when(pro_A_1<=0.5, alpha_A_1, pro_A_1>0.5);Fraction(add(signal_A_0, signal_A_1, filter = false))'},
 {'type': 'REGULAR',
  'settings': {'in

In [ ]:
result = ace.simulate_alpha_list_multi(s, alpha_list)

Warning, list of alphas too short, single concurrent simulations will be used instead of multisimulations


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

In [111]:
result

[{'alpha_id': 'Vp55RVY',
  'simulate_data': {'type': 'REGULAR',
   'settings': {'instrumentType': 'EQUITY',
    'region': 'GLB',
    'universe': 'MINVOL1M',
    'delay': 1,
    'decay': 0,
    'neutralization': 'MARKET',
    'truncation': 0.01,
    'pasteurization': 'ON',
    'testPeriod': 'P0Y0M0D',
    'unitHandling': 'VERIFY',
    'nanHandling': 'OFF',
    'language': 'FASTEXPR',
    'visualization': False},
   'regular': 'rank(add(trade_when(oth460_1l_ccns<=0.5, vector_neut(ts_zscore(ts_backfill(anl69_cpss_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_ccns>0.5), trade_when(oth460_1l_acda<=0.5, vector_neut(ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_acda>0.5), filter = false))'},
  'is_stats':         pnl  bookSize  longCount  shortCount  turnover  returns  drawdown  \
  0  12442248  20000000        962         962    0.2594   0.1201      0.06   
  
       margin  fitness  sha

In [112]:
#prettify_result function can be used from the helpful_functions library to take a look at IS stats of all the simulated alphas

result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,start_date,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover
0,12552083,20000000,962,962,0.2645,0.1212,0.0555,0.000917,1.59,2.35,2012-01-22,roVVn3j,"rank(add(trade_when(oth460_1l_acda<=0.5, regre...",PASS,PASS,FAIL,PASS,PASS,PASS,PASS
1,12442248,20000000,962,962,0.2594,0.1201,0.0600,0.000926,1.49,2.19,2012-01-22,Vp55RVY,"rank(add(trade_when(oth460_1l_ccns<=0.5, vecto...",PASS,PASS,FAIL,PASS,PASS,PASS,PASS


In [ ]:
result_st1.to_csv('A_mixing_add.csv')

### mult

In [99]:
'''
signal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));

signal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));

signal1 * (1 + signal2)
'''

'\nsignal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));\n\nsignal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));\n\nsignal1 * (1 + signal2)\n'

In [102]:
alpha_list = []

for i in range(10):
    alpha_list.append(ace.generate_alpha(f'{a[2*i]}*(1+ {a[2*i+1]})', region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.01))

alpha_list

[{'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 0,
   'neutralization': 'MARKET',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': 'trade_when(oth460_1l_acda<=0.5, regression_neut(ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_acda>0.5)*(1+ trade_when(oth460_1l_ccns<=0.5, regression_neut(ts_zscore(ts_backfill(anl69_cpss_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_ccns>0.5))'},
 {'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 0,
   'neutralization': 'MARKET',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VE

In [103]:
result = ace.simulate_alpha_list_multi(s, alpha_list)

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [22:51<00:00, 342.85s/it]


### corr

In [58]:
alpha_list = []

for i in range(10):
    alpha_list.append(ace.generate_alpha(f'k = ts_corr({a[i]}, {a[i+1]}, 30);k*regression_neut({a[i]}, {a[i+1]})', region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.01))

alpha_list



[{'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 0,
   'neutralization': 'MARKET',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': 'k = ts_corr(trade_when(oth460_1l_acda<=0.5, regression_neut(ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_acda>0.5), trade_when(oth460_1l_ccns<=0.5, regression_neut(ts_zscore(ts_backfill(anl69_cpss_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_ccns>0.5), 30);k*regression_neut(trade_when(oth460_1l_acda<=0.5, regression_neut(ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_acda>0.5), trade_when(oth460_1l_ccns<=0.5, regression_neut(ts_zscore(ts_backfill(

In [63]:
result = ace.simulate_alpha_list_multi(s, alpha_list)

 25%|████████████████████▊                                                              | 1/4 [03:07<09:22, 187.53s/it]

An error occurred


 50%|█████████████████████████████████████████▌                                         | 2/4 [04:58<04:45, 142.54s/it]

An error occurred


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [05:34<01:33, 93.81s/it]

An error occurred


100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [19:01<00:00, 285.34s/it]

An error occurred
Your simulation has been running too long. If you are running simulations in batch, consider to reduce number of concurrent simulations.


In [105]:
result

[{'alpha_id': 'RpKKkxa',
  'simulate_data': {'type': 'REGULAR',
   'settings': {'instrumentType': 'EQUITY',
    'region': 'GLB',
    'universe': 'MINVOL1M',
    'delay': 1,
    'decay': 0,
    'neutralization': 'MARKET',
    'truncation': 0.01,
    'pasteurization': 'ON',
    'unitHandling': 'VERIFY',
    'nanHandling': 'OFF',
    'language': 'FASTEXPR',
    'visualization': False},
   'regular': 'trade_when(oth460_1l_m3r_mtn_spb_se<=0.5, vector_neut(ts_zscore(ts_backfill(anl69_cps_best_eeps_cur_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_m3r_mtn_spb_se>0.5)*(1+ trade_when(oth460_1l_fbbs<=0.5, vector_neut(ts_zscore(ts_backfill(anl69_cpss_best_eeps_nxt_yr, 21),21), group_zscore(ts_ir(returns, 252), industry)), oth460_1l_fbbs>0.5))'},
  'is_stats':        pnl  bookSize  longCount  shortCount  turnover  returns  drawdown  \
  0  5634660  20000000        570        1060    0.6559   0.0544    0.1309   
  
       margin  fitness  sharpe   startDate alpha_id  
  0  0.

In [106]:
#prettify_result function can be used from the helpful_functions library to take a look at IS stats of all the simulated alphas

result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,start_date,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover
0,7457729,20000000,569,1283,0.6729,0.0720,0.0921,0.000214,0.39,1.20,2012-01-22,vL66dAQ,"trade_when(oth460_1l_m3r_mtn_spb_se<=0.5, vect...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS
1,7219126,20000000,813,1885,0.6375,0.0697,0.1073,0.000219,0.39,1.17,2012-01-22,5weeMrJ,"trade_when(oth460_1l_fbds<=0.5, vector_neut(ts...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS
2,7029477,20000000,688,1413,0.6486,0.0679,0.0926,0.000209,0.38,1.18,2012-01-22,Xpee1Nx,"trade_when(oth460_es_ebit_fy1_d1m_l1<=0.5, reg...",PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS
3,7170607,20000000,568,1299,0.6671,0.0692,0.0998,0.000208,0.38,1.18,2012-01-22,dxooQdv,"trade_when(oth460_1l_m3r_mtn_spb_se<=0.5, regr...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS
4,5795140,20000000,579,1344,0.5480,0.0560,0.0858,0.000204,0.33,1.02,2012-01-22,27WWKaZ,"trade_when(oth460_1l_acda<=0.5, regression_neu...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS
5,5656042,20000000,942,1866,0.5911,0.0546,0.1160,0.000185,0.30,1.00,2012-01-22,AwmmOpw,"trade_when(oth460_es_ebit_fy1_d1m_l1<=0.5, vec...",PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS
6,5490512,20000000,588,1337,0.5688,0.0530,0.1044,0.000186,0.29,0.95,2012-01-22,6wJJEzO,"trade_when(oth460_1l_ccns<=0.5, vector_neut(ts...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS
7,5620094,20000000,587,1159,0.5987,0.0543,0.1137,0.000181,0.27,0.89,2012-01-22,vL66NAd,"trade_when(oth460_es_roe_fy1_d1m_l1<=0.5, vect...",PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS
8,5634660,20000000,570,1060,0.6559,0.0544,0.1309,0.000166,0.26,0.91,2012-01-22,RpKKkxa,"trade_when(oth460_1l_m3r_mtn_spb_se<=0.5, vect...",PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS
9,4842543,20000000,574,1318,0.5459,0.0468,0.1084,0.000171,0.26,0.88,2012-01-22,Pp88XLL,"trade_when(oth460_1l_acda<=0.5, regression_neu...",PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS


In [62]:
result_st1.to_csv('A_mixing_add.csv')

In [113]:
#to take a look at the combined result of all new alphas

is_tests_df = hf.concat_is_tests(result)
is_tests_df.head()

,alpha_id,endDate,limit,name,result,startDate,value,year
0,Vp55RVY,NaN,1.58,LOW_SHARPE,PASS,NaN,2.1900,NaN
1,Vp55RVY,NaN,1.00,LOW_FITNESS,PASS,NaN,1.4900,NaN
2,Vp55RVY,NaN,0.01,LOW_TURNOVER,PASS,NaN,0.2594,NaN
3,Vp55RVY,NaN,0.70,HIGH_TURNOVER,PASS,NaN,0.2594,NaN
4,Vp55RVY,NaN,NaN,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN


In [114]:
#making a list of failed alphas
failed_alphas = is_tests_df.query('result=="FAIL"')['alpha_id'].unique()

#making a list of passed alphas
passed_alphas = list(set(is_tests_df['alpha_id']).difference(failed_alphas))

print(f'Failed alphas:{failed_alphas}\nPassed alphas:{passed_alphas}')

Failed alphas:['Vp55RVY' 'roVVn3j']
Passed alphas:[]


In [60]:
#calling submit_alpha on all alphas that have passed the submission tests
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['summer_boot_camp_4조'])

In [59]:
ace.get_check_submission(s, passed_alphas[3])

,name,result,limit,value,competitions,alpha_id
0,LOW_SHARPE,PASS,1.25,2.6200,NaN,roV9mgo
1,LOW_FITNESS,PASS,1.00,1.5400,NaN,roV9mgo
2,LOW_TURNOVER,PASS,0.01,0.4315,NaN,roV9mgo
3,HIGH_TURNOVER,PASS,0.70,0.4315,NaN,roV9mgo
4,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN,roV9mgo
5,LOW_SUB_UNIVERSE_SHARPE,PASS,0.92,1.2600,NaN,roV9mgo
6,SELF_CORRELATION,FAIL,0.70,0.8117,NaN,roV9mgo
7,PROD_CORRELATION,PENDING,0.70,NaN,NaN,roV9mgo
8,REGULAR_SUBMISSION,PASS,4.00,0.0000,NaN,roV9mgo
9,MATCHES_COMPETITION,PASS,NaN,NaN,"[{'id': 'challenge', 'name': 'Challenge'}]",roV9mgo


In [ ]:
submit_result = {alpha_id: ace.submit_alpha(s, alpha_id) for alpha_id in passed_alphas}

## Template C Mixing

In [83]:
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['summer_boot_camp_4조'])

In [87]:
ace.get_check_submission(s, passed_alphas[0])

,name,result,limit,value,competitions,alpha_id
0,LOW_SHARPE,PASS,1.25,2.0200,NaN,oKP2Wrl
1,LOW_FITNESS,PASS,1.00,1.1200,NaN,oKP2Wrl
2,LOW_TURNOVER,PASS,0.01,0.3259,NaN,oKP2Wrl
3,HIGH_TURNOVER,PASS,0.70,0.3259,NaN,oKP2Wrl
4,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN,oKP2Wrl
5,LOW_SUB_UNIVERSE_SHARPE,PASS,0.87,0.9200,NaN,oKP2Wrl
6,SELF_CORRELATION,PASS,0.70,0.4739,NaN,oKP2Wrl
7,PROD_CORRELATION,PASS,0.70,0.6252,NaN,oKP2Wrl
8,REGULAR_SUBMISSION,PASS,4.00,3.0000,NaN,oKP2Wrl
9,MATCHES_COMPETITION,PASS,NaN,NaN,"[{'id': 'challenge', 'name': 'Challenge'}]",oKP2Wrl


When you got a list of submittable alphas, you can call function submit_alpha()

In [89]:
#calling submit_alpha on all alphas that have passed the submission tests
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['summer_boot_camp_4조'])
submit_result = {alpha_id: ace.submit_alpha(s, alpha_id) for alpha_id in passed_alphas}

In [90]:
#submit_result will have return values from the submit_result function

submit_result

{'oKP2Wrl': True}

### Library Fuctions.

following are some other functions that you can use for your own analysis

**get_alpha_pnl(s, alpha_id)** - to get the pnl for an alpha

**get_alpha_yearly_stats(s, alpha_id)** - to get yearly statistics for an alpha

**get_self_corr(s, alpha_id)** - to get self correlation results for an alpha

**get_prod_corr(s, alpha_id)** - to get prod correlation results for an alpha

**get_check_submission(s, alpha_id)** - to get check submission result for an alpha

**check_self_corr_test(s, alpha_id)** - to check if alpha passes self correlation test (self_corr<0.7)

**check_prod_corr_test(s, alpha_id)** - to check if alpha passes prod correlation test (prod_corr<0.7)

**perfomance_comparison(s, alpha_id)** - to get the result of performance comparison for an alpha merged performance

In [37]:
ace.get_alpha_pnl(s, result[0]['alpha_id'])

,Pnl,alpha_id
Date,,
2012-01-23,0.0,dx9ALqJ
2012-01-24,52190.0,dx9ALqJ
2012-01-25,32038.0,dx9ALqJ
2012-01-26,37403.0,dx9ALqJ
2012-01-27,53631.0,dx9ALqJ
...,...,...
2022-01-17,12935472.0,dx9ALqJ
2022-01-18,12935661.0,dx9ALqJ
2022-01-19,12967474.0,dx9ALqJ
